# Análise do Arquivo Players
Este notebook é dedicado à implementação e análise de um modelo de classificação utilizando o algoritmo XGBoost para prever a probabilidade de jogadores marcarem gols com base em suas características, como desempenho do jogador, números de gols e outras métricas. O objetivo deste notebook é fornecer uma ferramenta para previsões, que pode ser usada para previsões de resultados.

# Objetivo
O objetivo desse notebook é fornecer previsões se um jogador marcará um gol em uma partida. Através de análise dos dados e do treinamento de um mondelo XGBoost, buscando identificar padrões e caractéristicas que mais influenciam o desempenho dos jogadores.

# Como usar Este Notebook
1. **Configuração do Ambiente:**

- **Google Colab:** No Google Colab, será necessário fazer o upload das tabelas para o Google Drive e montar o drive no notebook.
- **Localmente:** Se for rodar o notebook localmente, é necessário baixar as tabelas e colocá-las no mesmo diretório do notebook ou ajustar os caminhos dos arquivos conforme necessário.
2. **Instalação de Dependências:**

- Certifique-se de que todas as bibliotecas necessárias estão instaladas. Você pode instalar as dependências utilizando o seguinte comando:

 !pip install -r requirements.txt

3. **Execução do Notebook:**
- Siga as células de código sequencialmente para garantir que todas as etapas sejam executadas corretamente. Cada seção do notebook está organizada para facilitar a compreensão e a análise dos dados.

# Nesse Notebook Será Abordado
1. **Preparação e Treinamento de Dados:**
- Definição da variável target com base no número de gols e seleção das features a partir do dataset, removendo colunas desnecessárias.
- Divisão dos dados em conjunto de treino e teste, e padronização das variáveis para manter a mesma escala.
- Implementação do modelo XGBoost para classificar a probabilidade de um jogador marcar gol.

2. **Implementação de Função de Previsão:**

- Desenvolvimento de uma função que permite prever a probabilidade de um jogador específico marcar um gol com base em seu nome.

3. **Avaliação do Modelo:**
- Cálculo de métricas de desempenho, como acurácia e acurácia balanceada, além da exibição de um relatório de classificação detalhado.
- Visualização da matriz de confusão para melhor compreensão das previsões do modelo.
# Dependências
Para rodar o notebook de forma local, é recomendado que inicie uma venv (ambiente virtual) e instale as dependências.
Em algumas máquinas o xgboost não funicionará, pois ele é muito robusto. Então, caso isso aconteça use o Google Colab

Se estiver utilizando o Google Colab, pule esta etapa.



In [11]:
!pip install xgboost

In [12]:
!pip show xgboost


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, balanced_accuracy_score
import matplotlib.pyplot as plt
import xgboost as xgb
import seaborn as sns

# Carregando o dataset
Feita a importação do arquivo para leitura dos dados.

In [2]:
df = pd.read_csv('../data/tratado/players_tratado.csv')

# 1. Preparação e Treinamento de Dados

Nesta seção, os dados são preparados para o modelo de classificação. Isso inclui a seleção das features e da variável alvo (target), o tratamento de dados ausentes, a padronização das variáveis e o treinamento do modelo XGBoost.

#1.1 Seleção de Features
O objetivo é prever se um jogador marcará um gol com base no número de gols totais (goals_overall). A variável alvo foi definida como 1 (marcou gol) e 0 (não marcou gol), a partir da coluna goals_overall. Colunas irrelevantes, como jogadores que não entraram em campo e outras caractéristicas, foram removidas, pois não seriam não úteis para o modelo.

In [3]:
colunas_para_eliminar = ['sm_goals_scored_total_overall','rank_in_club_top_scorer', 'goals_involved_per_90_overall', 'goals_per_90_overall', 'min_per_goal_overall', 'goals_away', 'goals_home']
df = df.drop(columns=colunas_para_eliminar)

# Filtra o DataFrame para remover os jogadores com aparições em casa e fora de casa igual a 0
df = df[(df['appearances_home'] + df['appearances_away']) > 0]

# 1.2 Variável alvo
Definimos uma coluna target que classifica jogadores como 1 (se fizeram gols) ou 0 (se não fizeram). Em seguida, as features relevantes são selecionadas, enquanto informações como o número total de gols e o nome do jogador são removidas para evitar viés. Os dados são divididos em conjuntos de treino e teste, e as variáveis são padronizadas para garantir consistência nas escalas. O modelo XGBoost é treinado utilizando validação cruzada para calcular o Log Loss e ajustar o modelo. Por fim, o modelo é utilizado para fazer previsões com base nos dados de treino.

In [ ]:
# Supondo que 'df' seja o DataFrame contendo os dados

# Definir a coluna alvo (target) com base no número de gols
df['target'] = df['goals_overall'].apply(lambda x: 1 if x > 0 else 0)

# Definir as features (X) e a variável target (y)
# Remover 'goals_overall' para evitar que o modelo fique dependente dessa informação direta
X = df.drop(columns=['goals_overall', 'full_name', 'target', ])  # Remover colunas desnecessárias
y = df['target']

# Dividir os dados em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Padronizar os dados com a mesma escala
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # Aplica a padronização nos dados de treino
X_test_scaled = scaler.transform(X_test)  # Aplica a mesma padronização nos dados de teste

# Treinar o modelo XGBoost com validação cruzada
model = xgb.XGBClassifier(
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss',  # Você também pode testar 'auc' como métrica alternativa
    alpha=10,  # Regularização L1
    lambda_=10,  # Regularização L2
    subsample=0.8,  # Usar 80% dos dados para cada árvore
    colsample_bytree=0.5,  # Usar 80% das features para cada árvore
    max_depth=3  # Limitar a profundidade das árvores para evitar overfitting
)

# Implementar validação cruzada (cv = número de folds)
scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='neg_log_loss')

# Exibir os resultados da validação cruzada
print(f"Log Loss Médio: {-scores.mean():.4f}")
print(f"Desvio Padrão do Log Loss: {scores.std():.4f}")

# Ajustar o modelo usando todos os dados de treino
model.fit(X_train_scaled, y_train)
# Função para prever a probabilidade de um jogador marcar um gol usando o nome do jogador

# 2. Implementação de Função de Previsão
Nessa seção, desenvolvemos uma função```predict_player_goal prevê a probabilidade ``` de um jogador marcar um gol com base em um modelo treinado. Ela recebe o nome do jogador, verifica se ele está no dataset e prepara seus dados removendo colunas irrelevantes como goals_overall e full_name. Os dados do jogador são então padronizados usando o mesmo StandardScaler aplicado no treino. Por fim, a função usa o modelo XGBoost para calcular a probabilidade de gol e retorna essa informação em porcentagem.
<br>
<br>

**Desenvolvimento da Função predict_player_goal(player_name)**


A função ```predict_player_goal``` tem como objetivo calcular a probabilidade de um jogador marcar um gol. Ela segue os seguintes passos:
<br>


1. **Verificação da Existência do Jogador:** A função primeiro verifica se o nome do jogador fornecido existe no dataset ```(df['full_name'])```. Se o jogador não for encontrado, uma mensagem de erro é retornada.

2. **Seleção e Preparação dos Dados do Jogador:** Após confirmar que o jogador está presente no dataset, os dados referentes a ele são extraídos e algumas colunas irrelevantes, como ```goals_overall``` e ```full_name```, são removidas. Caso a coluna ```target``` também esteja presente, ela é excluída para manter a consistência com o modelo treinado.

3. **Padronização dos Dados:** Assim como os dados de treino foram padronizados, os dados do jogador também precisam passar pelo mesmo processo de padronização utilizando o ```StandardScaler``` previamente ajustado.

4. **Previsão de Probabilidade:** Após a padronização, os dados do jogador são alimentados no modelo XGBoost, que retorna a probabilidade de o jogador marcar um gol. A probabilidade é então formatada e exibida como uma porcentagem.

In [ ]:
# Função para prever a probabilidade de um jogador marcar um gol usando o nome do jogador
def predict_player_goal(player_name):
    global scaler, model
    # Verifica se o nome do jogador existe
    if player_name not in df['full_name'].values:
        return f"Jogador com nome {player_name} não encontrado."

    # Seleciona os dados do jogador com base no nome
    player_data = df[df['full_name'] == player_name].copy()

    # Remover a coluna 'goals_overall' caso esteja presente, pois ela foi removida durante o treinamento
    columns_to_drop = ['goals_overall', 'full_name']
    if 'target' in player_data.columns:
        columns_to_drop.append('target')

    # Remove colunas desnecessárias
    player_data = player_data.drop(columns=columns_to_drop)

    # Padronização dos dados
    player_data_scaled = scaler.transform(player_data)

    # Previsão da probabilidade de marcar um gol
    probabilidade = model.predict_proba(player_data_scaled)[:, 1]
    return f"A probabilidade do jogador {player_name} marcar um gol é de {probabilidade[0] * 100:.2f}%."

# Exemplo de uso
player_name = "Agustín Canobbio Graviz" # fez gol
player_name2 = 'Abner Salles da Silva'  # n fez gol
player_name3 = 'Adriano Martins da Fonseca' # n fez
player_name4 = 'Alan Javier Franco'
print(predict_player_goal(player_name))
print(predict_player_goal(player_name2))
print(predict_player_goal(player_name3))
print(predict_player_goal(player_name4))

# 3. Avaliação do Modelo
Após o treinamento, é essencial avaliar o desempenho do modelo para verificar sua eficácia na classificação correta dos jogadores que marcaram gols. Nesta seção, calculamos métricas de desempenho como acurácia, acurácia balanceada, precisão, recall e F1-score. Além disso, visualizamos a matriz de confusão para entender as previsões do modelo. Dessa forma, ao analisar as métricas conseguimos avaliar de forma eficiente a execução do modelo, oque pode nos revelar se o conjunto de dados é satisfatório ou não para a tarefa proposta. Se o desempenho do modelo não for adequado, pode ser necessário realizar ajustes, como a coleta de mais dados, o balanceamento das classes ou a otimização de hiperparâmetros. Com esses insights, é possível aprimorar o modelo ou refinar a abordagem adotada, garantindo resultados mais precisos e confiáveis no futuro.
<br>
<br>

**Cálculo das Métricas de Desempenho:**

Utilizamos as previsões do modelo no conjunto de teste para calcular as principais métricas de desempenho. Essas métricas ajudam a entender a precisão do modelo e como ele lida com os falsos positivos e falsos negativos.

In [ ]:
# Previsões no conjunto de teste
y_pred = model.predict(X_test)
# Calcular a acurácia
accuracy = accuracy_score(y_test, y_pred)
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
# Exibir as métricas
print(f"Acurácia do modelo: {accuracy * 100:.2f}%")
print(f"Acurácia balanceada do modelo: {balanced_accuracy * 100:.2f}%")
print("Relatório de Classificação:\n", classification_report(y_test, y_pred))


# 4. Visualização da Matriz de Confusão
Este código gera uma matriz de confusão para visualizar o desempenho do modelo de classificação em termos de suas previsões. Utilizando o Seaborn, uma biblioteca de visualização de dados do Python, o código cria um heatmap que exibe a matriz de confusão, onde as previsões corretas e incorretas do modelo são claramente indicadas. A matriz mostra a contagem de classificações corretas (verdadeiros positivos e negativos) e incorretas (falsos positivos e negativos) para as classes "Positivo" e "Negativo".

In [ ]:
# Matriz de Confusão
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Negativo', 'Positivo'], yticklabels=['Negativo', 'Positivo'])
plt.xlabel('Previsão')
plt.ylabel('Real')
plt.title('Matriz de Confusão')
plt.show()

# 5. Referências
Está é uma seção de referências com relação as bibliotecas que utilizamos ao longo deste arquivo

NUMPY. NumPy Documentation. Disponível em: https://numpy.org/doc/.

‌PANDAS. pandas documentation. Disponível em: https://pandas.pydata.org/docs/.

MATPLOTLIB. Matplotlib: Python plotting — Matplotlib 3.3.4 documentation. Disponível em: https://matplotlib.org/stable/index.html.

‌SCIKIT-LEARN. scikit-learn: machine learning in Python. Disponível em: https://scikit-learn.org/stable/.

‌SEABORN. seaborn: statistical data visualization — seaborn 0.9.0 documentation. Disponível em: https://seaborn.pydata.org/.